<a href="https://colab.research.google.com/github/the-cmyk/SandBox/blob/master/yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!wget https://github.com/ultralytics/yolov5/releases/download/v3.1/yolov5x.pt


In [2]:
pip install -U opencv-python pillow pyyaml tqdm

Requirement already up-to-date: opencv-python in /usr/local/lib/python3.6/dist-packages (4.4.0.46)
Requirement already up-to-date: pillow in /usr/local/lib/python3.6/dist-packages (8.0.1)
Requirement already up-to-date: pyyaml in /usr/local/lib/python3.6/dist-packages (5.3.1)
Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.51.0)


In [3]:
!wget https://daily.jstor.org/wp-content/uploads/2017/12/traffic_jam_1050x700.jpg

--2020-11-13 08:45:48--  https://daily.jstor.org/wp-content/uploads/2017/12/traffic_jam_1050x700.jpg
Resolving daily.jstor.org (daily.jstor.org)... 23.185.0.2, 2620:12a:8001::2, 2620:12a:8000::2
Connecting to daily.jstor.org (daily.jstor.org)|23.185.0.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 704907 (688K) [image/jpeg]
Saving to: ‘traffic_jam_1050x700.jpg.1’

traffic_jam_1050x70 100%[===================>] 688.39K  --.-KB/s    in 0.06s   

2020-11-13 08:45:48 (10.4 MB/s) - ‘traffic_jam_1050x700.jpg.1’ saved [704907/704907]



In [13]:
import torch
from PIL import Image
import numpy as np
from PIL import Image, ImageDraw

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l',pretrained=True).fuse().eval()  # yolov5s.pt
model = model.autoshape()  # for autoshaping of PIL/cv2/np inputs and NMS

# Images
img = Image.open('traffic_jam_1050x700.jpg')  # PIL image
imgs = [img, img, img]

# Inference
with torch.no_grad():
    prediction = model(imgs, size=640)  # includes NMS

for i, (img, pred) in enumerate(zip(imgs, prediction)):
    str = 'Image %g/%g: %gx%g ' % (i + 1, len(imgs), *img.shape[:2])
    img = Image.fromarray(img.astype(np.uint8)) if isinstance(img, np.ndarray) else img  # from np
    if pred is not None:
        for c in pred[:, -1].unique():
            n = (pred[:, -1] == c).sum()  # detections per class
            str += '%g %ss, ' % (n, model.names[int(c)])  # add to string
        for *box, conf, cls in pred:  # xyxy, confidence, class
            label = model.names[int(cls)] if hasattr(model, 'names') else 'class_%g' % cls
            # str += '%s %.2f, ' % (label, conf)  # label
            ImageDraw.Draw(img).rectangle(box, width=3)  # plot
    img.save('results%g.jpg' % i)  # save
    print(str + 'Done.')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      7040  models.common.Focus                     [3, 64, 3]                    
  1                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  2                -1  1    161152  models.common.BottleneckCSP             [128, 128, 3]                 
  3                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  4                -1  1   1627904  models.common.BottleneckCSP             [256, 256, 9]                 
  5                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  6                -1  1   6499840  models.common.BottleneckCSP             [512, 512, 9]                 
  7                -1  1   4720640  models.common.Conv                   

Fusing layers... 


Model Summary: 236 layers, 4.77901e+07 parameters, 4.77901e+07 gradients


Adding autoShape... 
Image 1/3: 700x1050 2 persons, 33 cars, 1 motorcycles, 5 buss, 8 trucks, Done.
Image 2/3: 700x1050 2 persons, 33 cars, 1 motorcycles, 5 buss, 8 trucks, Done.
Image 3/3: 700x1050 2 persons, 33 cars, 1 motorcycles, 5 buss, 8 trucks, Done.
